In [17]:
from utils import *
import re
from pprint import pprint
from nltk.corpus import stopwords

all_data = get_data_from_db()
all_data = all_data[(all_data["labels"]== "PO") | (all_data["labels"]== "NG")]

num_remover = NumRemover()
word_remover = WordRemover(stopwords.words('portuguese'))
all_data = num_remover.fit_transform(all_data)
all_data = word_remover.fit_transform(all_data)

neg_data = all_data[all_data["labels"] == "NG"]
pos_data = all_data[all_data["labels"] == "PO"]

ratio = 0.7
neg_train = neg_data.iloc[0:round(ratio*neg_data.shape[0]), :]
pos_train = pos_data.iloc[0:round(ratio*pos_data.shape[0]), :]

neg_test = neg_data.iloc[round(ratio*neg_data.shape[0]): , :]
pos_test = pos_data.iloc[round(ratio*neg_data.shape[0]): , :]

stopwords_pt = stopwords.words("portuguese")

# Vocabulários

In [18]:
from nltk import FreqDist
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer

vocab_neg = set(' '.join(neg_train["texts"].tolist()).split())
vocab_pos = set(' '.join(pos_train["texts"].tolist()).split())

print("Dimensão vocabulario negativo: " + str(len(vocab_neg)))
print("Dimensão vocabulario positivo: " + str(len(vocab_pos)))

Dimensão vocabulario negativo: 2687
Dimensão vocabulario positivo: 2326


In [19]:
# Verificar interseção dos vocabulários
intersect = vocab_neg.intersection(vocab_pos)
vocab_neg = vocab_neg.difference(intersect)
vocab_pos = vocab_pos.difference(intersect)

lexicon = list(vocab_neg.union(vocab_pos))

In [23]:
from sklearn.naive_bayes  import MultinomialNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

features = FeatureUnion([
                    ("bigram", CountVectorizer(ngram_range=(2,2), stop_words= stopwords_pt, binary= True)),
                    ("lexicon_vector", CountVectorizer(vocabulary= lexicon))
                    ])

print("MultinomialNB")
run_cross_validation(all_data, features, MultinomialNB(), n_folds=5)

print("\n\nSVM")
run_cross_validation(all_data, features, SVC(C=316), n_folds=5)

MultinomialNB
Cross Validation:
Fold  0  - Acuracia:  0.822695035461
Fold  1  - Acuracia:  0.778571428571
Fold  2  - Acuracia:  0.807142857143
Fold  3  - Acuracia:  0.785714285714
Fold  4  - Acuracia:  0.807142857143

Accuracia media:  0.800253292806
Desvio padrão:  0.0160003123144


SVM
Cross Validation:
Fold  0  - Acuracia:  0.723404255319
Fold  1  - Acuracia:  0.742857142857
Fold  2  - Acuracia:  0.785714285714
Fold  3  - Acuracia:  0.742857142857
Fold  4  - Acuracia:  0.707142857143

Accuracia media:  0.740395136778
Desvio padrão:  0.0263072323206
